<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
#импортируем библиотеки
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

from sklearn.utils import shuffle

In [2]:
#откроем файл и изучим его содержимое
df = pd.read_csv('/datasets/Churn.csv')
display(df.head())
df.info()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Согласно документации, в датасете имеются следующие признаки:
* RowNumber — индекс строки в данных
* CustomerId — уникальный идентификатор клиента
* Surname — фамилия
* CreditScore — кредитный рейтинг
* Geography — страна проживания
* Gender — пол
* Age — возраст
* Tenure — сколько лет человек является клиентом банка
* Balance — баланс на счёте
* NumOfProducts — количество продуктов банка, используемых клиентом
* HasCrCard — наличие кредитной карты
* IsActiveMember — активность клиента
* EstimatedSalary — предполагаемая зарплата

In [3]:
# удалим данные, которые нам не потребуются
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [4]:
df.isnull().mean()

CreditScore        0.0000
Geography          0.0000
Gender             0.0000
Age                0.0000
Tenure             0.0909
Balance            0.0000
NumOfProducts      0.0000
HasCrCard          0.0000
IsActiveMember     0.0000
EstimatedSalary    0.0000
Exited             0.0000
dtype: float64

In [5]:
# посмотрим наблюдения с пустыми значениями в столбце 'Tenure'
df[df['Tenure'].isnull()].head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
30,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
82,543,France,Female,36,NaN,0.00,2,0,0,26019.59,0
85,652,Spain,Female,75,NaN,0.00,2,1,1,114675.75,0
94,730,Spain,Male,42,NaN,0.00,2,0,1,85982.47,0
99,413,France,Male,34,NaN,0.00,2,0,0,6534.18,0
111,538,Germany,Male,39,NaN,108055.10,2,1,0,27231.26,0


Пропуски обнаружены лишь в столбце Tenure. Учитывая, что пользователи, для которых не заполнен признак о том, сколько лет он является клиентом банка, могут быть активными пользователями (IsActiveMember=1), иметь кредитные карты (HasCrCard=1) и пользоваться другими продуктами банка (NumOfProducts>1), можно сделать вывод, что пропуски не являются преднамеренными. Поэтому мы заполним их медианными значениями в зависимости от возраста клиента.

In [6]:
# замена пропусков
df['Tenure'] = df['Tenure'].fillna(df.groupby('Age')['Tenure'].transform('median'))

In [7]:
# проверим на наличие дубликатов
df.duplicated().sum()

0

In [8]:
# изменим тип данных тех признаков, что выражаются в деньгах
df = df.astype({'EstimatedSalary': 'int64', 'Balance': "int64"})

In [9]:
# Выделим целевую переменную и признаки
features = df.drop('Exited', axis=1)
target = df['Exited']

# Разделим исходную выборку на обучающую и валидационную, на обучающую выбоку выделим 60% выборки
x_train, x_test, y_train, y_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

# Поделим тестовую выборку на 2 части, чтобы создать валидационную выборку
x_test, x_valid, y_test, y_valid = train_test_split(
    x_test, y_test, test_size=0.5, random_state=12345)

# оценим размеры полученных выборок
display(x_train.shape)
display(x_test.shape)
x_valid.shape

(6000, 10)

(2000, 10)

(2000, 10)

In [10]:
# Создадим функцию для кодировки категориальных факторов методом OHE
def onehot_preprocessing(df, columns, ohe, mode='train'):
    """df- dataframe передаем весь датасет и выбираем столбцы"""
    if mode == 'train':
        x_tr = ohe.fit_transform(df[columns]).toarray()
    else:
        x_tr = ohe.transform(df[columns]).toarray()
    
    x_tr = pd.DataFrame(x_tr, columns=ohe.get_feature_names())
    x_res = pd.concat([df.drop(columns=columns).reset_index(), x_tr], axis=1)
    
    return x_res

In [11]:
# Закодируем факторы 'Geography' и 'Gender' на всех 3-х выборках
ohe = OneHotEncoder()
x_train = onehot_preprocessing(x_train, ['Geography', 'Gender'], ohe)
x_valid = onehot_preprocessing(x_valid, ['Geography', 'Gender'], ohe, mode='')
x_test = onehot_preprocessing(x_test, ['Geography', 'Gender'], ohe, mode='')

In [12]:
# оценим разброс значений признаков
df.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,4.995800,76485.571500,1.530200,0.70550,0.515100,100089.748400,0.203700
std,96.653299,10.487806,2.762118,62397.185107,0.581654,0.45584,0.499797,57510.491042,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.000000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51001.750000,0.000000
50%,652.000000,37.000000,5.000000,97198.000000,1.000000,1.00000,1.000000,100193.500000,0.000000
75%,718.000000,44.000000,7.000000,127643.750000,2.000000,1.00000,1.000000,149387.750000,0.000000
max,850.000000,92.000000,10.000000,250898.000000,4.000000,1.00000,1.000000,199992.000000,1.000000


In [13]:
# стандартизируем признаки, чтобы признаки, выраженные в деньгах, не получили бОльший вес в модели
col = ['CreditScore', 'Age', 'Tenure', 'Balance']
scaler = StandardScaler()

x_train[col] = scaler.fit_transform(x_train[col])
x_valid[col] = scaler.transform(x_valid[col])
x_test[col] = scaler.transform(x_test[col])

## Исследование задачи

Перед нами стоит задача классификации. В качестве таргета будет выступать переменная 'Exited'.
Исследуем баланс классов.

In [14]:
# оценим распределение переменной Exited
df['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

Выборка явно не сбалансирована - тех, что кто ушел из банка, почти в 4 раза меньше чем общее количество текущих клиентов

In [15]:
# оценим уровни корреляции между всеми признаками
x_train.corr()

,index,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,x0_France,x0_Germany,x0_Spain,x1_Female,x1_Male
index,1.000000,0.007851,-0.010482,-0.001973,-0.010269,-0.001393,0.009845,0.014714,-0.000239,0.017591,-0.003094,-0.017215,-0.015112,0.015112
CreditScore,0.007851,1.000000,-0.013542,0.000036,0.010169,0.012745,-0.010165,0.022480,0.001702,-0.001175,0.003270,-0.001898,-0.002881,0.002881
Age,-0.010482,-0.013542,1.000000,-0.027636,0.029278,-0.033108,-0.000300,0.093706,-0.006184,-0.045237,0.044103,0.008313,0.027948,-0.027948
Tenure,-0.001973,0.000036,-0.027636,1.000000,-0.014801,0.011637,0.029588,-0.026949,0.003925,0.008108,-0.005819,-0.003564,-0.012513,0.012513
Balance,-0.010269,0.010169,0.029278,-0.014801,1.000000,-0.307142,-0.021450,-0.032714,0.021291,-0.233750,0.396732,-0.125023,-0.005972,0.005972
NumOfProducts,-0.001393,0.012745,-0.033108,0.011637,-0.307142,1.000000,-0.002674,0.026512,0.008146,0.011899,-0.013251,-0.000544,0.022021,-0.022021
HasCrCard,0.009845,-0.010165,-0.000300,0.029588,-0.021450,-0.002674,1.000000,-0.019424,-0.018114,-0.013280,0.016744,-0.001337,0.004772,-0.004772
IsActiveMember,0.014714,0.022480,0.093706,-0.026949,-0.032714,0.026512,-0.019424,1.000000,-0.010344,-0.000269,-0.030380,0.030534,-0.028302,0.028302
EstimatedSalary,-0.000239,0.001702,-0.006184,0.003925,0.021291,0.008146,-0.018114,-0.010344,1.000000,-0.018350,0.019144,0.002124,0.010491,-0.010491
x0_France,0.017591,-0.001175,-0.045237,0.008108,-0.233750,0.011899,-0.013280,-0.000269,-0.018350,1.000000,-0.575344,-0.581276,-0.020278,0.020278


Самый высокий уровень корреляции: 1)между признаками "количество продуктов" и "балансом", однако было решено оставить оба признака, так как они могут по-разному объяснять целевую переменную 2)между объемом средств на балансе и страной=Германия, однако это, скорее, говорит об уровне благосостояния в Германии 

Для начала построим 3 модели на обучающей выборке и подберем гиперпараменты на валидационной выборке без учета дисбаланса классов

In [1]:
def find_best_model(x_train, y_train, x_valid, y_valid, model, params):
    """
    Функция для поиска лучших гиперпараметров для заданной модели на основе GridSearchCV.
    Возвращает лучшую модель и соответствующий ей f1-score на валидационной выборке.
    """
    grid_search = GridSearchCV(model, params, cv=5, scoring='f1')
    grid_search.fit(x_train, y_train)

    # Вывод лучших гиперпараметров и f1-score
    best_params = grid_search.best_params_
    best_model = grid_search.best_estimator_
    best_f1_score = grid_search.best_score_
    print('Best hyperparameters:', best_params)
    print('Best F1 score:', '{:.2f}'.format(best_f1_score))

    # Оценка лучшей модели на валидационной выборке
    predictions = best_model.predict(x_valid)
    best_f1_score_val = f1_score(y_valid, predictions)
    print('Validation set F1 score:', '{:.2f}'.format(best_f1_score_val))
    
    return best_model, best_f1_score_val

In [18]:
# Поиск лучших параметров для модели Decision Tree Classifier
params = {'max_depth': range(3, 50)}

dt = DecisionTreeClassifier(random_state=12345)

best_model, best_f1_score = find_best_model(x_train, y_train, x_valid, y_valid, dt, params)

Best hyperparameters: {'max_depth': 7}
Best F1 score: 0.56
Validation set F1 score: 0.50


In [19]:
# Поиск лучших параметров для логистической регресии
params = {'solver': ['liblinear', 'lbfgs'], 'max_iter': range(100, 2000, 10)}

lr = LogisticRegression(random_state=12345)

best_model, best_f1_score = find_best_model(x_train, y_train, x_valid, y_valid, lr, params)

Best hyperparameters: {'max_iter': 100, 'solver': 'liblinear'}
Best F1 score: 0.00
Validation set F1 score: 0.00


In [20]:
# Поиск лучших параметров для модели случайного леса
params = {'n_estimators': [50, 100, 150],'max_depth': range(3, 50)}

rf = RandomForestClassifier(random_state=12345)

best_model, best_f1_score = find_best_model(x_train, y_train, x_valid, y_valid, rf, params)

Best hyperparameters: {'max_depth': 19, 'n_estimators': 100}
Best F1 score: 0.57
Validation set F1 score: 0.51


После подбора гиперпараметров выберем модель с наибольшим f-score. Проверять эффективность модели будет на тестовой выборке

In [24]:
# Лучшие модели на валидационном датасете
model_1 = DecisionTreeClassifier(max_depth=7, random_state=12345)
model_2 = RandomForestClassifier(max_depth=19, n_estimators= 100, random_state=12345)

In [25]:
# функция для тестирования модели на тестовой выборке
def model_test(x_train, y_train, x_test, y_test, model):
    model.fit(x_train, y_train)
    predictions = model.predict(x_test)
    print("F1-score:", '{:.2f}'.format(f1_score(y_test, predictions)))
    
    probabilities_test = model.predict_proba(x_test)
    probabilities_one_valid = probabilities_test[:, 1]
    print("AUC-ROC:", '{:.2f}'.format(roc_auc_score(y_test, probabilities_one_valid)))

In [26]:
# тестирование модели Decision Tree Classifier
model_test(x_train, y_train, x_test, y_test, model_1)

F1-score: 0.55
AUC-ROC: 0.81


In [27]:
# тестирование модели Random Forest Classifier
model_test(x_train, y_train, x_test, y_test, model_2)

F1-score: 0.56
AUC-ROC: 0.84


Лучшей моделью на тестовой выборке оказалась модель случайного леса, поскольку у нее самый высокий показатель f1-меры (0.56)

## Борьба с дисбалансом

Построим модели с учетом дисбаланса классов. Для этого используем разные подходы

In [31]:
# построение логистической регрессии с учетом дисбаланса класса
best_model = None
best_result = 0
for iter in range(100, 2000):
    model = LogisticRegression(
        random_state=12345, solver='liblinear', max_iter=iter, class_weight='balanced')
    model.fit(x_train, y_train)
    predictions = model.predict(x_valid)
    result = f1_score(y_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        
print(best_result, best_model)

0.48484848484848475 LogisticRegression(class_weight='balanced', random_state=12345,
                   solver='liblinear')


Балансировка классов улучшила показатель f1-score с 0 до 0.48

In [32]:
# функция для балансировки классов при помощи техники upsample
def upsample(features, target, repeat):
    features_zeros = features[target == 0].reset_index(drop=True)
    features_ones = features[target == 1].reset_index(drop=True)
    target_zeros = target[target == 0].reset_index(drop=True)
    target_ones = target[target == 1].reset_index(drop=True)

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
features_upsampled, target_upsampled = upsample(x_train, y_train, 4)

In [33]:
# функция для балансировки классов при помощи техники upsample
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_downsampled, target_downsampled = downsample(x_train, y_train, 0.25)

In [35]:
# Поиск лучших параметров для модели Decision Tree Classifier
params = {'max_depth': range(3, 50)}

dt = DecisionTreeClassifier(random_state=12345)

best_model, best_f1_score = find_best_model(features_upsampled, target_upsampled, x_valid, y_valid, dt, params)
best_model, best_f1_score = find_best_model(features_downsampled, target_downsampled, x_valid, y_valid, dt, params)

Best hyperparameters: {'max_depth': 22}
Best F1 score: 0.93
Validation set F1 score: 0.45
Best hyperparameters: {'max_depth': 6}
Best F1 score: 0.74
Validation set F1 score: 0.59


Балансировка классов на основе техники upsample улучшила f1-score с 0.56 до 0.93 на тренировочной выборке и ухудшила с 0.50 до 0.45 - на валидационной.  
Балансировка классов на основе техники downsample улучшила f1-score с 0.56 до 0.74 на тренировочной выборке и с 0.50 до 0.59 - на валидационной.  
*Поскольку выбор лучшей модели и техники балансировки мы будем делать, исходя из результатов валидационной выборки, тестировать на тестовой выборке будем модель с параметрами {'max_depth': 6}, которая строилась на выборке, сбалансированной при помощи техники downsample*

In [36]:
# Поиск лучших параметров для модели случайного леса
params = {'n_estimators': [50, 100, 150],'max_depth': range(3, 50)}

rf = RandomForestClassifier(random_state=12345)

best_model, best_f1_score = find_best_model(features_upsampled, target_upsampled, x_valid, y_valid, rf, params)
best_model, best_f1_score = find_best_model(features_downsampled, target_downsampled, x_valid, y_valid, rf, params)

Best hyperparameters: {'max_depth': 21, 'n_estimators': 100}
Best F1 score: 0.96
Validation set F1 score: 0.59
Best hyperparameters: {'max_depth': 6, 'n_estimators': 150}
Best F1 score: 0.77
Validation set F1 score: 0.57


Балансировка классов на основе техники upsample улучшила f1-score с 0.57 до 0.96 на тренировочной выборке и с 0.51 до 0.59 - на валидационной.  
Балансировка классов на основе техники downsample улучшила f1-score с 0.57 до 0.77 на тренировочной выборке и с 0.51 до 0.57 - на валидационной.  
*Поскольку выбор лучшей модели и техники балансировки мы будем делать, исходя из результатов валидационной выборки, тестировать будем модель с гиперпараметрами {'max_depth': 21, 'n_estimators': 100}, которая строилась на выборке, сбалансированной при помощи техники upsample*

## Тестирование модели

In [37]:
# лучшие модели на тестовой выборке
model_dt_bal = DecisionTreeClassifier(max_depth=6, random_state=12345)
model_rf_bal = RandomForestClassifier(max_depth=21, n_estimators= 100, random_state=12345)

In [38]:
# тестирование модели Decision Tree Classifier
model_test(features_downsampled, target_downsampled, x_test, y_test, model_dt_bal)

F1-score: 0.58
AUC-ROC: 0.83


In [39]:
# тестирование модели Random Forest Classifier
model_test(features_upsampled, target_upsampled, x_test, y_test, model_rf_bal)

F1-score: 0.61
AUC-ROC: 0.83


Наилучшая модель - модель случайного леса, с учетом балансировки классов с помощью техники upsample значение F1-меры достигло 0.61 и AUC-ROC - 0.83 на тестовой выборке. 

## Вывод

Для целей прогнозирования уйдет клиент из банка или нет было построено несколько моделей без учета того, что выборки не сбалансированы (в исторических данных, на которых модель обучалась, тех, кто ушел в 4 раза меньше), а также - с учетом. Балансировка классов позволила значительно улучшить качество всех моделей. Наиболее эффективно ранжирующей моделью по результатам тестирования была выбрана модель случайного леса, которая обучалась на выборке, сбалансированной при помощи техники upsample. Ее значение F1-меры  достигло 0.61 и AUC-ROC - 0.83 на тестовой выборке. 